In [68]:
import sqlite3

import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter

In [69]:
SAVE = True

In [70]:
conn = sqlite3.connect("backup/data.db")
q = "SELECT * FROM posts"
df = pd.read_sql_query(q, conn)
df.head()

,index,angry,created_time,haha,id,like,love,message,parent_id,sad,source,tagged_user_id,tagged_user_name,tagged_user_type,total_reactions,type,wow
0,0,0.0,2018-02-14T17:50:00+0000,2.0,561380070875128_570517729961362,3.0,0.0,"#1298 כשאני רעב ואין לי כסף, אני הולך למכונת מ...",None,1.0,TAU,None,None,None,6.0,POST,0.0
1,1,0.0,2018-02-14T17:40:00+0000,0.0,561380070875128_570517519961383,1.0,0.0,#1297 עד מתי ימשיכו הסטודנטים להנדסה מכנית עם ...,None,0.0,TAU,None,None,None,1.0,POST,0.0
2,2,0.0,2018-02-14T17:30:00+0000,0.0,561380070875128_570517376628064,7.0,0.0,#1296 ניגשתי לתקופת המבחנים הזו בידיעה שאני הו...,None,1.0,TAU,None,None,None,8.0,POST,0.0
3,3,0.0,2018-02-14T17:20:00+0000,7.0,561380070875128_570516906628111,15.0,0.0,"#1295 יוצא לשירותים באמצע המבחן, אחרי 5 דקות ה...",None,0.0,TAU,None,None,None,22.0,POST,0.0
4,4,0.0,2018-02-14T17:10:00+0000,11.0,561380070875128_570516283294840,42.0,2.0,"#1294 מישהי פעם התחילה איתי מיד אחרי ההרצאה, ו...",None,2.0,TAU,None,None,None,58.0,POST,1.0


In [71]:
df.groupby(["type"]).agg(["count"])["index"]

,count
type,
COMMENT,52584
POST,7843
TAG,43316


In [80]:
u_df = df.groupby(["source"]).agg(["count"])["index"].reset_index().sort_values(["count"])
u_df

,source,count
10,SALSA,108
13,YIZRAEL,188
9,MTA,372
8,IDF,722
3,BRAUDE,1486
5,HAIFA,1803
4,HADASSAH,2629
2,BIU,5253
7,IDC,5377
0,ARIEL,6309


Extract manual post id from the posts:

In [73]:
ID_REGEX = "(#\d+|\d+#){1}"
df["id"] = df["message"].str.findall(ID_REGEX).str.get(0)
df["message"].str
df["reply_to"] = df["message"].str.findall(ID_REGEX).str.get(1)
df["message"] = df["message"].str.replace(ID_REGEX, "")
df.head()

,index,angry,created_time,haha,id,like,love,message,parent_id,sad,source,tagged_user_id,tagged_user_name,tagged_user_type,total_reactions,type,wow,reply_to
0,0,0.0,2018-02-14T17:50:00+0000,2.0,#1298,3.0,0.0,"כשאני רעב ואין לי כסף, אני הולך למכונת ממתקים...",None,1.0,TAU,None,None,None,6.0,POST,0.0,NaN
1,1,0.0,2018-02-14T17:40:00+0000,0.0,#1297,1.0,0.0,עד מתי ימשיכו הסטודנטים להנדסה מכנית עם רגשי ...,None,0.0,TAU,None,None,None,1.0,POST,0.0,NaN
2,2,0.0,2018-02-14T17:30:00+0000,0.0,#1296,7.0,0.0,ניגשתי לתקופת המבחנים הזו בידיעה שאני הולכת ל...,None,1.0,TAU,None,None,None,8.0,POST,0.0,NaN
3,3,0.0,2018-02-14T17:20:00+0000,7.0,#1295,15.0,0.0,"יוצא לשירותים באמצע המבחן, אחרי 5 דקות הדודה ...",None,0.0,TAU,None,None,None,22.0,POST,0.0,NaN
4,4,0.0,2018-02-14T17:10:00+0000,11.0,#1294,42.0,2.0,"מישהי פעם התחילה איתי מיד אחרי ההרצאה, והיא ח...",None,2.0,TAU,None,None,None,58.0,POST,1.0,NaN


Mark question posts:

In [74]:
df.loc[df["message"].str.find("?") > 0, "is_question"] = True
df["is_question"] = df["is_question"].fillna(False)
df.head()

,index,angry,created_time,haha,id,like,love,message,parent_id,sad,source,tagged_user_id,tagged_user_name,tagged_user_type,total_reactions,type,wow,reply_to,is_question
0,0,0.0,2018-02-14T17:50:00+0000,2.0,#1298,3.0,0.0,"כשאני רעב ואין לי כסף, אני הולך למכונת ממתקים...",None,1.0,TAU,None,None,None,6.0,POST,0.0,NaN,False
1,1,0.0,2018-02-14T17:40:00+0000,0.0,#1297,1.0,0.0,עד מתי ימשיכו הסטודנטים להנדסה מכנית עם רגשי ...,None,0.0,TAU,None,None,None,1.0,POST,0.0,NaN,True
2,2,0.0,2018-02-14T17:30:00+0000,0.0,#1296,7.0,0.0,ניגשתי לתקופת המבחנים הזו בידיעה שאני הולכת ל...,None,1.0,TAU,None,None,None,8.0,POST,0.0,NaN,False
3,3,0.0,2018-02-14T17:20:00+0000,7.0,#1295,15.0,0.0,"יוצא לשירותים באמצע המבחן, אחרי 5 דקות הדודה ...",None,0.0,TAU,None,None,None,22.0,POST,0.0,NaN,True
4,4,0.0,2018-02-14T17:10:00+0000,11.0,#1294,42.0,2.0,"מישהי פעם התחילה איתי מיד אחרי ההרצאה, והיא ח...",None,2.0,TAU,None,None,None,58.0,POST,1.0,NaN,True


Creates a clean message column:

In [75]:
df['clean_message'] = df['message'].str.replace("[^א-ת ]", "")
df['clean_message'] = df['clean_message'].str.replace("[\s]", " ")

Export analyzed df to sql:

In [76]:
if SAVE:
    df.to_sql("posts", sqlite3.connect("analyzed_data.db"), if_exists="replace")

In [77]:
all_messages_cat = df["clean_message"].str.cat(sep="")
words_counter = Counter(all_messages_cat.split(" "))
del words_counter[""]

In [78]:
words_counter_df = pd.DataFrame.from_dict(words_counter, orient='index').reset_index()
words_counter_df.head()

,index,0
0,כשאני,332
1,רעב,16
2,ואין,177
3,לי,5939
4,כסף,202


In [79]:
list(words_counter.keys())[:10]

['כשאני',
 'רעב',
 'ואין',
 'לי',
 'כסף',
 'אני',
 'הולך',
 'למכונת',
 'ממתקים',
 'דוחף']

#### Get the replies for each comment

#### Get the name of the commenter (for each comment or reply)

#### Get the commenter gender

#### Get the posts a user reacted to or commented to

#### Get a list of the users commented or reacted

#### Calculate the feeling in each institute